In [1]:
from osgeo import ogr, gdal
import os
import subprocess
import rasterio


import numpy as np
import glob

In [2]:
image_path = '/home/shrayank_mistry/Modules/data/raster.tif' 
mask_path  = '/home/shrayank_mistry/Modules/data/mask.tif'

In [3]:
data = rasterio.open(image_path)
print(data.bounds)

extent = data.bounds

# col_add, row_add = 109800, 109800
col_add, row_add = 40960, 40960 

left, bottom, right, top = extent[0], extent[3], extent[0], extent[3]
print(left, bottom, right, top)

BoundingBox(left=300000.0, bottom=1890240.0, right=409800.0, top=2100000.0)
300000.0 2100000.0 300000.0 2100000.0


In [4]:
raster_patches = '/home/shrayank_mistry/Modules/data/main_test/raster/'
mask_patches = '/home/shrayank_mistry/Modules/data/main_test/mask/'

In [5]:
p = 1
for i in range(1):
    for j in range(1):
        l, b, r, t = left + (col_add * j), bottom - (row_add * i), right + (col_add * (j + 1)), top - (row_add * (i + 1))
        
        cmd = f'gdal_translate -projwin {l} {b} {r} {t} -of GTiff {image_path} {raster_patches}{p}.tif'
        subprocess.call(cmd.split())
        p = p + 1

In [6]:
p = 1
for i in range(1):
    for j in range(1):
        l, b, r, t = left + (col_add * j), bottom - (row_add * i), right + (col_add * (j + 1)), top - (row_add * (i + 1))
        
        cmd = f'gdal_translate -projwin {l} {b} {r} {t} -ot Byte -of GTiff {mask_path} {mask_patches}{p}.tif'
        subprocess.call(cmd.split())
        p = p + 1

In [7]:
def _16bit_to_8Bit_raster(inputRaster, outputRaster, outputPixType='Byte', outputFormat='png', percentiles=[2, 98]):

   srcRaster = gdal.Open(inputRaster)
   cmd = ['gdal_translate', '-ot', outputPixType, '-of', 
          outputFormat]

   for bandId in range(srcRaster.RasterCount):
       bandId = bandId+1
       band = srcRaster.GetRasterBand(bandId)

       bmin = band.GetMinimum()        
       bmax = band.GetMaximum()
       # if not exist minimum and maximum values
       if bmin is None or bmax is None:
           (bmin, bmax) = band.ComputeRasterMinMax(1)
       # else, rescale
       band_arr_tmp = band.ReadAsArray()
       bmin = np.percentile(band_arr_tmp.flatten(), 
                           percentiles[0])
       bmax= np.percentile(band_arr_tmp.flatten(), 
                           percentiles[1])

       cmd.append('-scale_{}'.format(bandId))
       cmd.append('{}'.format(bmin))
       cmd.append('{}'.format(bmax))
       cmd.append('{}'.format(0))
       cmd.append('{}'.format(255))

   cmd.append(inputRaster)
   cmd.append(outputRaster)
#    print("Conversin command:", cmd)
   subprocess.call(cmd)

In [8]:
raster_patches = '/home/shrayank_mistry/Modules/data/main_test/raster/'
raster_files = os.listdir(raster_patches)

In [9]:
for file in raster_files:
    resimPath = raster_patches + file

    dstPath   = '/home/shrayank_mistry/Modules/data/main_test/raster_png/'
    # dstPath = '/home/shrayank_mistry/Modules/val_dir/val/'
    dstPath   = dstPath + '1.' + "png"

    # print(dstPath)

    _16bit_to_8Bit_raster(resimPath, dstPath, outputPixType = 'Float32', outputFormat = "png")